In [9]:
# Import dependencies
import datetime
import json
import time
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import copy
import textwrap
import requests

# Load environment variables
load_dotenv("./.env")

# Define function for calling GPT4o
def gpt4o_call(system_message, prompt):
    client = AzureOpenAI(
        api_version=os.getenv("4oAPI_VERSION"),
        azure_endpoint=os.getenv("4oAZURE_ENDPOINT"),
        api_key=os.getenv("4oAPI_KEY")
    )

    start_time = time.time()

    completion = client.chat.completions.create(
        model=os.getenv("4oMODEL"),
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    result = json.loads(completion.model_dump_json(indent=2))
    completion_text = result["choices"][0]["message"]["content"]

    return completion_text

# Define function for calling O1
def o1_call(system_message, user_message):
    prompt = system_message + user_message

    url = os.getenv("o1AZURE_ENDPOINT")
    headers = {
        "api-key": os.getenv("o1API_KEY"),
        "Content-Type": "application/json"
    }
    data = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt}
                ],
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    messageo1 = response.json()["choices"][0]["message"]["content"]
    
    return messageo1

In [5]:
system_message="You are a helpful AI assistant."

user_message="""
hi
"""

completion_text_4o=gpt4o_call(system_message,user_message)
print(completion_text_4o)

completion_text_o1 = o1_call(system_message, user_message)
print(completion_text_o1)

Hello! How can I assist you today?


Hello! How can I assist you today?


In [20]:
# Example of an OpenAI ChatCompletion request with stream=True
# https://platform.openai.com/docs/api-reference/streaming#chat/create-stream
client = AzureOpenAI(
    api_version=os.getenv("4oAPI_VERSION"),
    azure_endpoint=os.getenv("4oAZURE_ENDPOINT"),
    api_key=os.getenv("4oAPI_KEY")
)
# a ChatCompletion request
response = client.chat.completions.create(
    model=os.getenv("4oMODEL"),
    messages=[
        {'role': 'user', 'content': "hi"}
    ],
    temperature=0,
    stream=True  # this time, we set stream=True
)

for chunk in response:
    print(chunk)
    if chunk.choices and chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content)

ChatCompletionChunk(id='', choices=[], created=0, model='', object='', service_tier=None, system_fingerprint=None, usage=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
ChatCompletionChunk(id='chatcmpl-ADAIPXk1labX03VI8fm6fNFju9nvE', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results={})], created=1727701205, model='gpt-4o-2024-08-06', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_67802d9a6d', usage=None)
ChatCompletionChunk(id='chatcmpl-ADAIPXk1labX03VI8fm6fNFju9nvE', choices=[Choice(delta=ChoiceDelta(content='Hello', function_call=None, refu